In [6]:
import requests
import json


def query_overpass_api(overpass_query, output_file='overpass_result.geojson', timeout=300):
    """
    Query Overpass API and save data as GeoJSON file
    
    Parameters:
    - overpass_query: Overpass QL query string
    - output_file: Output filename
    - timeout: Request timeout in seconds
    
    Returns:
    - GeoJSON data dictionary
    """
    
    # Overpass API endpoint
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    print(f"Executing Overpass query...")
    print(f"Query content:")
    print(overpass_query)
    print("-" * 50)
    
    try:
        # Send POST request
        response = requests.post(
            overpass_url,
            data={'data': overpass_query},
            timeout=timeout,
            headers={'User-Agent': 'Python-Overpass-Client'}
        )
        
        # Check response status
        if response.status_code == 200:
            # Parse JSON data
            data = response.json()
            
            print(f"Query successful!")
            print(f"Retrieved {len(data.get('elements', []))} elements")
            
            # Convert to GeoJSON format if needed
            if 'generator' in data and data['generator'] == 'Overpass API':
                # Already Overpass format, add necessary GeoJSON structure
                geojson_data = {
                    "type": "FeatureCollection",
                    "generator": "overpass-turbo", 
                    "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.",
                    "timestamp": data.get('timestamp_osm_base', ''),
                    "features": []
                }
                
                # Simple element to feature conversion (may need more complex processing)
                for element in data.get('elements', []):
                    if element.get('type') == 'way' and 'lat' in element.get('nodes', [{}])[0]:
                        # Process way type
                        if 'geometry' in element:
                            coords = [[node['lon'], node['lat']] for node in element['geometry']]
                            feature = {
                                "type": "Feature",
                                "properties": element.get('tags', {}),
                                "geometry": {
                                    "type": "Polygon" if element.get('tags', {}).get('area') == 'yes' else "LineString",
                                    "coordinates": [coords] if element.get('tags', {}).get('area') == 'yes' else coords
                                },
                                "id": f"{element['type']}/{element['id']}"
                            }
                            geojson_data['features'].append(feature)
                
                # If conversion fails, save original data
                if not geojson_data['features']:
                    geojson_data = data
                    
            else:
                geojson_data = data
            
            # Save to file
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(geojson_data, f, ensure_ascii=False, indent=2)
            
            print(f"Data saved to: {output_file}")
            return geojson_data
            
        else:
            print(f"Request failed: HTTP {response.status_code}")
            print(f"Error message: {response.text}")
            return None
            
    except requests.exceptions.Timeout:
        print(f"Request timeout ({timeout} seconds)")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Network error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        return None
    except Exception as e:
        print(f"Unknown error: {e}")
        return None

print("Overpass API query function loaded!")


Overpass API query function loaded!


In [7]:
# Some commonly used query templates
query_templates = {
    "squares": '''[out:json][timeout:240];
(
way["area"="yes"]["highway"="unclassified"]["place"="square"](if:is_closed());
relation["type"="multipolygon"]["highway"="unclassified"]["place"="square"];
);
(._;>;);
out;''',

    "restaurants": '''[out:json][timeout:25];
(
  node["amenity"="restaurant"]({{bbox}});
  way["amenity"="restaurant"]({{bbox}});
);
out center;''',
    
    "parks": '''[out:json][timeout:25];
(
  way["leisure"="park"]({{bbox}});
  relation["leisure"="park"]({{bbox}});
);
out geom;''',
    
    "schools": '''[out:json][timeout:25];
(
  node["amenity"="school"]({{bbox}});
  way["amenity"="school"]({{bbox}});
);
out center;''',
}

print("Available query templates:")
for name, query in query_templates.items():
    print(f"  - {name}")

print("\nUsage examples:")
print("1. Use template: query_overpass_api(query_templates['squares'], 'squares.geojson')")
print("2. Custom query: query_overpass_api(your_custom_query, 'output.geojson')")


Available query templates:
  - squares
  - restaurants
  - parks
  - schools

Usage examples:
1. Use template: query_overpass_api(query_templates['squares'], 'squares.geojson')
2. Custom query: query_overpass_api(your_custom_query, 'output.geojson')


In [8]:
# Interactive query function
def custom_overpass_query():
    """
    Interactive input for custom Overpass query
    """
    print("Custom Overpass query")
    print("Example query format:")
    print("""
[out:json][timeout:25];
(
  node["amenity"="restaurant"]({{bbox}});
  way["amenity"="restaurant"]({{bbox}});
  relation["amenity"="restaurant"]({{bbox}});
);
out center;
    """)
    
    print("\nPlease enter your Overpass query (input 'default' to use default query):")
    
    # Can manually input query or use default query
    use_default = input("Use default square query? (y/n): ").lower().strip()
    
    if use_default in ['y', 'yes', '']:
        query = query_templates['squares']
        print("Using default square query")
    else:
        print("Please paste your Overpass query (press Enter to finish):")
        query = input()
    
    filename = input("Output filename (default: custom_query.geojson): ").strip()
    if not filename:
        filename = 'custom_query.geojson'
    
    return query_overpass_api(query, filename)

print("Interactive query function loaded!")
print("Run custom_overpass_query() to start interactive query")


Interactive query function loaded!
Run custom_overpass_query() to start interactive query


In [9]:
custom_overpass_query()

Custom Overpass query
Example query format:

[out:json][timeout:25];
(
  node["amenity"="restaurant"]({{bbox}});
  way["amenity"="restaurant"]({{bbox}});
  relation["amenity"="restaurant"]({{bbox}});
);
out center;
    

Please enter your Overpass query (input 'default' to use default query):
Using default square query
Executing Overpass query...
Query content:
[out:json][timeout:240];
(
way["area"="yes"]["highway"="unclassified"]["place"="square"](if:is_closed());
relation["type"="multipolygon"]["highway"="unclassified"]["place"="square"];
);
/*added by auto repair*/
(._;>;);
/*end of auto repair*/
out;
--------------------------------------------------
Query successful!
Retrieved 173 elements
Data saved to: custom_query.geojson


{'version': 0.6,
 'generator': 'Overpass API 0.7.62.7 375dc00a',
 'osm3s': {'timestamp_osm_base': '2025-06-13T15:01:30Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 252933423,
   'lat': 46.5070126,
   'lon': 6.4976404},
  {'type': 'node', 'id': 252933424, 'lat': 46.5070098, 'lon': 6.4974203},
  {'type': 'node', 'id': 253209420, 'lat': 46.5068281, 'lon': 6.4974616},
  {'type': 'node', 'id': 259834282, 'lat': 43.8369927, 'lon': 10.6911191},
  {'type': 'node',
   'id': 259834283,
   'lat': 43.8373324,
   'lon': 10.6911602,
   'tags': {'crossing': 'zebra',
    'crossing:island': 'no',
    'highway': 'crossing',
    'tactile_paving': 'no'}},
  {'type': 'node', 'id': 286982585, 'lat': 44.1202107, 'lon': 9.7090318},
  {'type': 'node', 'id': 286983095, 'lat': 44.1204325, 'lon': 9.7087628},
  {'type': 'node', 'id': 517778651, 'lat': 44.1203683, 'lon': 9.709099},
  {'type': 'n